In [1]:
# Celda A: imports y utilidades
import os
from pathlib import Path
import re
import math
import pandas as pd
import numpy as np
from IPython.display import display, HTML

results_root = Path("results")  # ajusta si tus resultados están en otra carpeta

def extract_fold_from_name(filename):
    m = re.search(r"_fold_(\d+)\.csv$", str(filename))
    return int(m.group(1)) if m else None

def read_csv_safe(path):
    try:
        return pd.read_csv(path)
    except Exception as e:
        print(f"WARNING: no se pudo leer {path}: {e}")
        return pd.DataFrame()

def collect_map_dfs(ds_name):
    """Recolecta los CSV per-fold de MAP para un dataset y concatena."""
    ds_dir = results_root / ds_name
    pattern = ds_dir.glob(f"{ds_name}_mlp_best_configs_fold_*.csv")
    paths = sorted(pattern, key=lambda p: extract_fold_from_name(p) or 0)
    dfs = []
    for p in paths:
        df = read_csv_safe(p)
        if df.empty:
            continue
        # Añadir fold si falta (extraer del nombre)
        if 'fold' not in df.columns:
            fold = extract_fold_from_name(p)
            if fold is not None:
                df['fold'] = fold
        dfs.append(df)
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return pd.DataFrame()

def collect_post_dfs(ds_name):
    """Recolecta todos los CSV de métricas de post (todos los métodos) y concatena.
       Añade columnas 'method' y 'fold' inferidas de la ruta/nombre."""
    ds_dir = results_root / ds_name / "post"
    if not ds_dir.exists():
        return pd.DataFrame()
    files = list(ds_dir.rglob("*_metrics_fold_*.csv"))
    rows = []
    for p in sorted(files):
        df = read_csv_safe(p)
        if df.empty:
            continue
        # inferir método desde la ruta: results/{ds}/post/{method}/...
        parts = list(p.parts)
        method = "unknown"
        if 'post' in parts:
            idx = parts.index('post')
            if idx + 1 < len(parts):
                method = parts[idx + 1]
        # fold from filename
        fold = extract_fold_from_name(p)
        df['method'] = method
        if 'fold' not in df.columns:
            df['fold'] = fold
        rows.append(df)
    if rows:
        return pd.concat(rows, ignore_index=True)
    else:
        return pd.DataFrame()

def stderr_from_stdstd(std, n):
    """Devuelve error estándar (std / sqrt(n)), segura para n<=0."""
    if n <= 0:
        return np.nan
    return std / math.sqrt(n)


In [2]:
# Celda B: función que calcula y muestra los resúmenes MAP y Posterior para un dataset
def summarize_dataset(ds_name, show_dfs=True, save_aggregated_csv=True):
    ds_name_cap = ds_name  # asumes nombres con la capitalización como en tu árbol (Boston, Energy...)
    print(f"=== Dataset: {ds_name_cap} ===")
    # MAP
    map_df = collect_map_dfs(ds_name_cap)
    if map_df.empty:
        print("No se encontraron CSVs MAP para", ds_name_cap)
    else:
        # Ensure numeric columns exist and cast
        cols_median = ['fold','num_layers','hidden_units','weight_decay']
        cols_stats = ['test_rmse','test_mae','training_time']
        # Cast columnas numéricas si existen
        for c in cols_median + cols_stats:
            if c in map_df.columns:
                map_df[c] = pd.to_numeric(map_df[c], errors='coerce')

        n = len(map_df)
        # medianas
        medians = {c: (map_df[c].median() if c in map_df.columns else np.nan) for c in cols_median}
        # medias, std, stderr para stats
        stats = {}
        for c in cols_stats:
            if c in map_df.columns:
                mean = map_df[c].mean()
                std = map_df[c].std(ddof=1)  # muestra
                sem = stderr_from_stdstd(std, n)
            else:
                mean = std = sem = np.nan
            stats[c+'_mean'] = mean
            stats[c+'_std']  = std
            stats[c+'_sem']  = sem

        summary_map = {**medians, **stats, 'n_folds': n}
        summary_map_df = pd.DataFrame([summary_map])
        print("MAP summary (medianas y stats):")
        display(summary_map_df)

        if save_aggregated_csv:
            outp = results_root / ds_name_cap / f"{ds_name_cap}_map_summary.csv"
            summary_map_df.to_csv(outp, index=False)
            print("Guardado:", outp)

    # POST
    post_df = collect_post_dfs(ds_name_cap)
    if post_df.empty:
        print("No se encontraron CSVs POST para", ds_name_cap)
    else:
        # Buscar columnas NLL, CQM, CRPS (case-insensitive)
        found_cols = {}
        for desired in ['NLL','CQM','CRPS']:
            matches = [c for c in post_df.columns if c.lower() == desired.lower()]
            if matches:
                found_cols[desired] = matches[0]
            else:
                # buscar substring (por si vienen tipo 'nll' u 'Nll')
                matches = [c for c in post_df.columns if desired.lower() in c.lower()]
                found_cols[desired] = matches[0] if matches else None

        # agrupar por método
        methods = post_df['method'].unique()
        rows = []
        for m in sorted(methods):
            sub = post_df[post_df['method'] == m]
            n_m = len(sub)
            row = {'method': m, 'n_folds': n_m}
            for key, col in found_cols.items():
                if col is not None and col in sub.columns:
                    vals = pd.to_numeric(sub[col], errors='coerce').dropna()
                    mean = vals.mean() if len(vals) > 0 else np.nan
                    std = vals.std(ddof=1) if len(vals) > 1 else np.nan
                    sem = stderr_from_stdstd(std, len(vals)) if not np.isnan(std) else np.nan
                else:
                    mean = std = sem = np.nan
                row[f"{key}_mean"] = mean
                row[f"{key}_std"]  = std
                row[f"{key}_sem"]  = sem
            rows.append(row)
        summary_post_df = pd.DataFrame(rows)
        print("POST summary (por método):")
        display(summary_post_df)

        if save_aggregated_csv:
            outp2 = results_root / ds_name_cap / f"{ds_name_cap}_post_summary_by_method.csv"
            summary_post_df.to_csv(outp2, index=False)
            print("Guardado:", outp2)

    return {'map_df': map_df, 'post_df': post_df, 'map_summary': (summary_map_df if 'summary_map_df' in locals() else None), 'post_summary': (summary_post_df if 'summary_post_df' in locals() else None)}


In [3]:
datasets = ["Boston","Energy","Yacht","Concrete","RedWine"]
results_summary = {}
for ds in datasets:
    print("\n\n*******************************")
    out = summarize_dataset(ds, show_dfs=True, save_aggregated_csv=True)
    results_summary[ds] = out




*******************************
=== Dataset: Boston ===
MAP summary (medianas y stats):


,fold,num_layers,hidden_units,weight_decay,test_rmse_mean,test_rmse_std,test_rmse_sem,test_mae_mean,test_mae_std,test_mae_sem,training_time_mean,training_time_std,training_time_sem,n_folds
0,7.0,3.0,50.0,0.0,3.972077,0.928817,0.257607,2.714937,0.49462,0.137183,1.748593,0.5091,0.141199,13


Guardado: results/Boston/Boston_map_summary.csv
POST summary (por método):


,method,n_folds,NLL_mean,NLL_std,NLL_sem,CQM_mean,CQM_std,CQM_sem,CRPS_mean,CRPS_std,CRPS_sem
0,kron,13,2.765235,0.125479,0.034802,0.137756,0.052281,0.014500,2.094920,0.328504,0.091111
1,lla,13,2.729001,0.144839,0.040171,0.113782,0.055884,0.015499,2.058104,0.339190,0.094074
2,llla,13,2.879608,0.479953,0.133115,0.069350,0.034273,0.009506,2.048831,0.397691,0.110300
3,qla,13,2.727948,0.144490,0.040074,0.113507,0.056034,0.015541,2.057933,0.339283,0.094100


Guardado: results/Boston/Boston_post_summary_by_method.csv


*******************************
=== Dataset: Energy ===
MAP summary (medianas y stats):


...

Guardado: results/Concrete/Concrete_post_summary_by_method.csv


*******************************
=== Dataset: RedWine ===
MAP summary (medianas y stats):


,fold,num_layers,hidden_units,weight_decay,test_rmse_mean,test_rmse_std,test_rmse_sem,test_mae_mean,test_mae_std,test_mae_sem,training_time_mean,training_time_std,training_time_sem,n_folds
0,6.0,2.0,50.0,0.0,0.655417,0.025078,0.007561,0.511396,0.022735,0.006855,2.234441,0.264112,0.079633,11


Guardado: results/RedWine/RedWine_map_summary.csv
POST summary (por método):


,method,n_folds,NLL_mean,NLL_std,NLL_sem,CQM_mean,CQM_std,CQM_sem,CRPS_mean,CRPS_std,CRPS_sem
0,kron,11,1.012899,0.026997,0.008140,0.059836,0.022084,0.006658,0.368023,0.012521,0.003775
1,lla,11,1.002202,0.029558,0.008912,0.046685,0.018597,0.005607,0.366256,0.013150,0.003965
2,llla,11,0.997734,0.038896,0.011728,0.028455,0.007061,0.002129,0.364393,0.014254,0.004298
3,qla,11,1.001494,0.029617,0.008930,0.046001,0.018329,0.005526,0.366136,0.013177,0.003973


Guardado: results/RedWine/RedWine_post_summary_by_method.csv
